# 1. Importing modules and functions

In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.Chem import MACCSkeys
from copy import deepcopy
from rdkit.ML.Descriptors import MoleculeDescriptors
from molvs import standardize_smiles
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import joblib
import pickle
from numpy import savetxt
from padelpy import from_sdf
from IPython.display import HTML
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from padelpy import from_sdf
import shap
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
def convert_smi_to_canon_smi(smi):
    
    try:
        canon_smi = Chem.MolToSmiles(Chem.MolFromSmiles(smi),isomericSmiles = False)
    except:
        canon_smi='wrong_smiles'
    return canon_smi

In [3]:
def standart(smi):
    global m
    if smi!='wrong_smiles':
        try:
            smiles=standardize_smiles(smi)
            m = Chem.MolFromSmiles(smi)
        except:
            smi='error kekule'
    else:
        m = 'check the smiles'
    return m

## Load data and curation work set

In [4]:
# Set file path
df_ws=pd.read_csv('Acute Toxicity_rat_oral_LD50.csv')
df_ws

,TAID,Pubchem CID,IUPAC Name,SMILES,Canonical SMILES,InChIKey,rat_oral_LD50
0,TOX-145,785,"benzene-1,4-diol",Oc1ccc(O)cc1,Oc1ccc(O)cc1,QIGBRXMKCJKVMJ-UHFFFAOYSA-N,2.561828
1,TOX-245,5453,tris(aziridin-1-yl)-sulfanylidene-lambda5-phos...,S=P(N1CC1)(N1CC1)N1CC1,S=P(N1CC1)(N1CC1)N1CC1,FOCVUCIESVLUNU-UHFFFAOYSA-N,3.915248
2,TOX-1279,4091,"3-(diaminomethylidene)-1,1-dimethylguanidine",CN(C)C(=N)N=C(N)N,CN(C)C(=N)N=C(N)N,XZWYZXLIPXDOLR-UHFFFAOYSA-N,2.111152
3,TOX-1281,8784,3-phenylprop-2-enoic acid,O=C(O)C=Cc1ccccc1,O=C(O)C=Cc1ccccc1,WBYWAXJHAXSJNI-UHFFFAOYSA-N,1.772794
4,TOX-1282,10364,2-methyl-5-propan-2-ylphenol,Cc1ccc(C(C)C)cc1O,Cc1ccc(C(C)C)cc1O,RECUKUPTGUEGMW-UHFFFAOYSA-N,2.268246
...,...,...,...,...,...,...,...
9838,TOX-109776,3079285,"N-cyclohexyl-6-methyl-4-oxopyrido[1,2-a]pyrimi...",Cc1cccc2ncc(C(=O)NC3CCCCC3)c(=O)n12,Cc1cccc2ncc(C(=O)NC3CCCCC3)c(=O)n12,VOWAKWKKYNYQCV-UHFFFAOYSA-N,3.057433
9839,TOX-109792,14991,(1-methylpiperidin-3-yl) 2-cyclohexyl-2-hydrox...,CN1CCCC(OC(=O)C(O)(c2ccccc2)C2CCCCC2)C1,CN1CCCC(OC(=O)C(O)(c2ccccc2)C2CCCCC2)C1,LCFBCKSQWVQIBY-UHFFFAOYSA-N,2.675328
9840,TOX-109817,216337,"7-[3-[cyclohexyl(methyl)amino]propyl]-1,3-dime...",CN(CCCn1cnc2c1c(=O)n(C)c(=O)n2C)C1CCCCC1,CN(CCCn1cnc2c1c(=O)n(C)c(=O)n2C)C1CCCCC1,YSSDOJFBBPZKBV-UHFFFAOYSA-N,2.308434
9841,TOX-113361,3518,2-[2-(azocan-1-yl)ethyl]guanidine,N=C(N)NCCN1CCCCCCC1,N=C(N)NCCN1CCCCCCC1,ACGDKVXYNVEAGU-UHFFFAOYSA-N,2.276164


 Convert a SMILES string to canonical SMILES

In [5]:
df_ws1 = deepcopy(df_ws)
df_ws1["SMILES"] = df_ws1.apply(lambda x: convert_smi_to_canon_smi(x.SMILES), axis=1)
df_ws1

,TAID,Pubchem CID,IUPAC Name,SMILES,Canonical SMILES,InChIKey,rat_oral_LD50
0,TOX-145,785,"benzene-1,4-diol",Oc1ccc(O)cc1,Oc1ccc(O)cc1,QIGBRXMKCJKVMJ-UHFFFAOYSA-N,2.561828
1,TOX-245,5453,tris(aziridin-1-yl)-sulfanylidene-lambda5-phos...,S=P(N1CC1)(N1CC1)N1CC1,S=P(N1CC1)(N1CC1)N1CC1,FOCVUCIESVLUNU-UHFFFAOYSA-N,3.915248
2,TOX-1279,4091,"3-(diaminomethylidene)-1,1-dimethylguanidine",CN(C)C(=N)N=C(N)N,CN(C)C(=N)N=C(N)N,XZWYZXLIPXDOLR-UHFFFAOYSA-N,2.111152
3,TOX-1281,8784,3-phenylprop-2-enoic acid,O=C(O)C=Cc1ccccc1,O=C(O)C=Cc1ccccc1,WBYWAXJHAXSJNI-UHFFFAOYSA-N,1.772794
4,TOX-1282,10364,2-methyl-5-propan-2-ylphenol,Cc1ccc(C(C)C)cc1O,Cc1ccc(C(C)C)cc1O,RECUKUPTGUEGMW-UHFFFAOYSA-N,2.268246
...,...,...,...,...,...,...,...
9838,TOX-109776,3079285,"N-cyclohexyl-6-methyl-4-oxopyrido[1,2-a]pyrimi...",Cc1cccc2ncc(C(=O)NC3CCCCC3)c(=O)n12,Cc1cccc2ncc(C(=O)NC3CCCCC3)c(=O)n12,VOWAKWKKYNYQCV-UHFFFAOYSA-N,3.057433
9839,TOX-109792,14991,(1-methylpiperidin-3-yl) 2-cyclohexyl-2-hydrox...,CN1CCCC(OC(=O)C(O)(c2ccccc2)C2CCCCC2)C1,CN1CCCC(OC(=O)C(O)(c2ccccc2)C2CCCCC2)C1,LCFBCKSQWVQIBY-UHFFFAOYSA-N,2.675328
9840,TOX-109817,216337,"7-[3-[cyclohexyl(methyl)amino]propyl]-1,3-dime...",CN(CCCn1cnc2c1c(=O)n(C)c(=O)n2C)C1CCCCC1,CN(CCCn1cnc2c1c(=O)n(C)c(=O)n2C)C1CCCCC1,YSSDOJFBBPZKBV-UHFFFAOYSA-N,2.308434
9841,TOX-113361,3518,2-[2-(azocan-1-yl)ethyl]guanidine,N=C(N)NCCN1CCCCCCC1,N=C(N)NCCN1CCCCCCC1,ACGDKVXYNVEAGU-UHFFFAOYSA-N,2.276164


In [6]:
print('Original data: ', len(df_ws), 'molecules')
print('Failed data: ', len(df_ws1[df_ws1['SMILES']=='wrong_smiles']), 'molecules')

Original data:  9843 molecules
Failed data:  0 molecules


In [7]:
index=df_ws1.index[df_ws1['SMILES']=='wrong_smiles'].tolist()
wrong_smiles=df_ws.iloc[index]
wrong_smiles=wrong_smiles.SMILES
number=[x+1 for x in index]
bad_molecules = pd.DataFrame({'No. failed smiles in original set': number, 'SMILES of wrong structure: ': wrong_smiles}, index=None)
bad_molecules = bad_molecules.set_index('No. failed smiles in original set')
bad_molecules

,SMILES of wrong structure:
No. failed smiles in original set,


##  Standardization  for work set

In [8]:
df_ws1["Molecule"] = df_ws1.apply(lambda x: standart(x.SMILES), axis=1)
moldf_ws=df_ws1[df_ws1['SMILES']!='wrong_smiles']
print('Kept data: ', len(moldf_ws), 'molecules')

[18:13:30] Unusual charge on atom 8 number of radical electrons set to zero
[18:13:30] Unusual charge on atom 0 number of radical electrons set to zero
[18:13:30] Unusual charge on atom 16 number of radical electrons set to zero
[18:13:30] Unusual charge on atom 16 number of radical electrons set to zero


Kept data:  9843 molecules


In [9]:
moldf_ws

,TAID,Pubchem CID,IUPAC Name,SMILES,Canonical SMILES,InChIKey,rat_oral_LD50,Molecule
0,TOX-145,785,"benzene-1,4-diol",Oc1ccc(O)cc1,Oc1ccc(O)cc1,QIGBRXMKCJKVMJ-UHFFFAOYSA-N,2.561828,<rdkit.Chem.rdchem.Mol object at 0x00000130EFB...
1,TOX-245,5453,tris(aziridin-1-yl)-sulfanylidene-lambda5-phos...,S=P(N1CC1)(N1CC1)N1CC1,S=P(N1CC1)(N1CC1)N1CC1,FOCVUCIESVLUNU-UHFFFAOYSA-N,3.915248,<rdkit.Chem.rdchem.Mol object at 0x00000130EFB...
2,TOX-1279,4091,"3-(diaminomethylidene)-1,1-dimethylguanidine",CN(C)C(=N)N=C(N)N,CN(C)C(=N)N=C(N)N,XZWYZXLIPXDOLR-UHFFFAOYSA-N,2.111152,<rdkit.Chem.rdchem.Mol object at 0x00000130EFB...
3,TOX-1281,8784,3-phenylprop-2-enoic acid,O=C(O)C=Cc1ccccc1,O=C(O)C=Cc1ccccc1,WBYWAXJHAXSJNI-UHFFFAOYSA-N,1.772794,<rdkit.Chem.rdchem.Mol object at 0x00000130EFB...
4,TOX-1282,10364,2-methyl-5-propan-2-ylphenol,Cc1ccc(C(C)C)cc1O,Cc1ccc(C(C)C)cc1O,RECUKUPTGUEGMW-UHFFFAOYSA-N,2.268246,<rdkit.Chem.rdchem.Mol object at 0x00000130F0C...
...,...,...,...,...,...,...,...,...
9838,TOX-109776,3079285,"N-cyclohexyl-6-methyl-4-oxopyrido[1,2-a]pyrimi...",Cc1cccc2ncc(C(=O)NC3CCCCC3)c(=O)n12,Cc1cccc2ncc(C(=O)NC3CCCCC3)c(=O)n12,VOWAKWKKYNYQCV-UHFFFAOYSA-N,3.057433,<rdkit.Chem.rdchem.Mol object at 0x00000130EFD...
9839,TOX-109792,14991,(1-methylpiperidin-3-yl) 2-cyclohexyl-2-hydrox...,CN1CCCC(OC(=O)C(O)(c2ccccc2)C2CCCCC2)C1,CN1CCCC(OC(=O)C(O)(c2ccccc2)C2CCCCC2)C1,LCFBCKSQWVQIBY-UHFFFAOYSA-N,2.675328,<rdkit.Chem.rdchem.Mol object at 0x00000130EFD...
9840,TOX-109817,216337,"7-[3-[cyclohexyl(methyl)amino]propyl]-1,3-dime...",CN(CCCn1cnc2c1c(=O)n(C)c(=O)n2C)C1CCCCC1,CN(CCCn1cnc2c1c(=O)n(C)c(=O)n2C)C1CCCCC1,YSSDOJFBBPZKBV-UHFFFAOYSA-N,2.308434,<rdkit.Chem.rdchem.Mol object at 0x00000130EFD...
9841,TOX-113361,3518,2-[2-(azocan-1-yl)ethyl]guanidine,N=C(N)NCCN1CCCCCCC1,N=C(N)NCCN1CCCCCCC1,ACGDKVXYNVEAGU-UHFFFAOYSA-N,2.276164,<rdkit.Chem.rdchem.Mol object at 0x00000130EFD...


In [10]:
y_tr=moldf_ws.rat_oral_LD50
y_tr

0       2.561828
1       3.915248
2       2.111152
3       1.772794
4       2.268246
          ...   
9838    3.057433
9839    2.675328
9840    2.308434
9841    2.276164
9842    3.053026
Name: rat_oral_LD50, Length: 9843, dtype: float64

In [11]:
moldf_ws=moldf_ws.Molecule

## Calculation MACCS Fingerprints for work set

In [12]:
fp_tr = [MACCSkeys.GenMACCSKeys(m) for m in moldf_ws]

In [13]:
def rdkit_numpy_convert(fp_tr):
    output = []
    for f in fp_tr:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [14]:
from numpy import savetxt
x_tr = rdkit_numpy_convert(fp_tr)

In [15]:
savetxt('models/MACCS/x_tr_MACCS.csv', x_tr, delimiter=',')

In [15]:
x_tr.shape

(9843, 167)

 ## GradientBoostingRegressor model building and validation

In [16]:
seed = 42

In [17]:
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [19]:
param_grid = {'learning_rate': [0.02,0.05],
                  'subsample'    : [0.9, 0.5, 0.1],
                  'n_estimators' : [100,500,1000],
                  'max_depth'    : [4, 10]
                 }

In [20]:
m = GridSearchCV(GradientBoostingRegressor(), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [21]:
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.02, 0.05], 'max_depth': [4, 10],
                         'n_estimators': [100, 500, 1000],
                         'subsample': [0.9, 0.5, 0.1]},
             verbose=1)

In [22]:
best_GBR = m.best_estimator_

In [23]:
m.best_params_

{'learning_rate': 0.02, 'max_depth': 10, 'n_estimators': 500, 'subsample': 0.5}

In [19]:
y_pred_CV_GBR = cross_val_predict(best_GBR, x_tr, y_tr, cv=cv)

In [28]:
y_pred_CV_GBR

array([1.92275016, 3.47143708, 1.80988463, ..., 2.71148066, 1.82214237,
       2.59287673])

In [20]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_GBR), 2)
Q2_CV

0.59

In [21]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_GBR)), 2)
RMSE_CV

0.57

# save the model to disk

In [31]:
pickle.dump(best_GBR, open('models/MACCS/Toxicity_GBR_MACCS.pkl', 'wb'))

# load the model from disk

In [18]:
best_GBR = pickle.load(open('models/MACCS/Toxicity_GBR_MACCS.pkl', 'rb'))

# SVM model building and validation

In [16]:
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [17]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [18]:
svm = GridSearchCV(SVR(C=1.0, epsilon=0.2), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [19]:
svm.fit(x_tr, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVR(epsilon=0.2), n_jobs=-1,
             param_grid={'C': [1, 10, 100, 1000, 10000],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]},
             verbose=1)

In [20]:
svm.best_params_
best_svm = svm.best_estimator_

In [21]:
svm.best_params_

{'C': 1, 'gamma': 0.1}

In [23]:
y_pred_CV_svm = cross_val_predict(best_svm, x_tr, y_tr, cv=cv)

In [24]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_svm), 2)
Q2_CV

0.58

In [25]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_svm)), 2)
RMSE_CV

0.58

In [28]:
pickle.dump(best_svm, open('models/MACCS/Toxicity_SVM_MACCS.pkl', 'wb'))

load the model from disk

In [22]:
best_svm = pickle.load(open('models/MACCS/Toxicity_SVM_MACCS.pkl', 'rb'))

# Multi-layer Perceptron regressor

In [29]:
from sklearn.neural_network import MLPRegressor

In [30]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [31]:
param_grid ={"hidden_layer_sizes": [(400, 300, 200, 100),(100, 100, 100), (10, 10, 10),(50,)], "activation": ["tanh", "relu"], "solver": ["lbfgs", "sgd", "adam"], "alpha": [0.00005,0.0005], 'max_iter': [1000, 2000]}

In [32]:
m = GridSearchCV(MLPRegressor(), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [33]:
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=MLPRegressor(), n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [5e-05, 0.0005],
                         'hidden_layer_sizes': [(400, 300, 200, 100),
                                                (100, 100, 100), (10, 10, 10),
                                                (50,)],
                         'max_iter': [1000, 2000],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             verbose=1)

In [34]:
best_MLPR = m.best_estimator_

In [35]:
m.best_params_

{'activation': 'relu',
 'alpha': 0.0005,
 'hidden_layer_sizes': (400, 300, 200, 100),
 'max_iter': 1000,
 'solver': 'adam'}

In [27]:
y_pred_CV_MLPR = cross_val_predict(best_MLPR, x_tr, y_tr, cv=cv)

In [28]:
y_pred_CV_MLPR

array([2.04437118, 3.4102707 , 1.75513223, ..., 2.29719794, 1.92297545,
       2.54904246])

In [29]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_MLPR), 2)
Q2_CV

0.51

In [30]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_MLPR)), 2)
RMSE_CV

0.63

# save the model to disk

In [43]:
pickle.dump(best_MLPR, open('models/MACCS/Toxicity_MLPR_MACCS.pkl', 'wb'))

# load the model from disk

In [26]:
best_MLPR = pickle.load(open('models/MACCS/Toxicity_MLPR_MACCS.pkl', 'rb'))

# k-nearest neighbors

In [44]:
k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)

In [45]:
m = GridSearchCV(KNeighborsRegressor(), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [46]:
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30]},
             verbose=1)

In [47]:
best_kNN = m.best_estimator_

In [48]:
m.best_params_

{'n_neighbors': 4}

In [49]:
y_pred_ws_kNN = best_kNN.predict(x_tr)

In [32]:
y_pred_CV_kNN = cross_val_predict(best_kNN, x_tr, y_tr, cv=cv)

In [33]:
y_pred_CV_kNN

array([2.18633298, 3.46489608, 1.68553511, ..., 2.59506387, 2.12134473,
       2.76270729])

In [34]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_kNN), 2)
Q2_CV

0.52

In [35]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_kNN)), 2)
RMSE_CV

0.62

# save the model to disk

In [56]:
pickle.dump(best_kNN, open('models/MACCS/Toxicity_kNN_MACCS.pkl', 'wb'))

# load the model from disk

In [31]:
best_kNN = pickle.load(open('models/MACCS/Toxicity_kNN_MACCS.pkl', 'rb'))

# CatBoostRegressor

In [1]:
from catboost import CatBoostRegressor

In [48]:
cv=KFold(n_splits=5, random_state=42, shuffle=True)

In [56]:
%%time
model = CatBoostRegressor()
parameters = {'depth' : [6,8,10],
              'learning_rate' : [0.01, 0.05, 0.1],
              'iterations'    : [100,500, 1000]
              }

grid = GridSearchCV(estimator=model, param_grid = parameters, n_jobs=-1, cv = cv)
grid.fit(x_tr, y_tr, verbose=False)

CPU times: total: 8min 46s
Wall time: 16min 35s


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=<catboost.core.CatBoostRegressor object at 0x000001309A1660B0>,
             n_jobs=-1,
             param_grid={'depth': [6, 8, 10], 'iterations': [100, 500, 1000],
                         'learning_rate': [0.01, 0.05, 0.1]})

In [57]:
best_CatBR = grid.best_estimator_

In [58]:
grid.best_params_

{'depth': 10, 'iterations': 1000, 'learning_rate': 0.05}

In [59]:
%%time
y_pred_CV_CatBR = cross_val_predict(best_CatBR, x_tr, y_tr, cv=cv, verbose=False)

0:	learn: 0.8822294	total: 39.2ms	remaining: 39.2s
1:	learn: 0.8686730	total: 80.1ms	remaining: 40s
2:	learn: 0.8553217	total: 121ms	remaining: 40.4s
3:	learn: 0.8434570	total: 161ms	remaining: 40.2s
4:	learn: 0.8330485	total: 202ms	remaining: 40.2s
5:	learn: 0.8228053	total: 244ms	remaining: 40.4s
6:	learn: 0.8130063	total: 301ms	remaining: 42.8s
7:	learn: 0.8035049	total: 360ms	remaining: 44.6s
8:	learn: 0.7960727	total: 414ms	remaining: 45.5s
9:	learn: 0.7864501	total: 463ms	remaining: 45.9s
10:	learn: 0.7779197	total: 532ms	remaining: 47.8s
11:	learn: 0.7698230	total: 582ms	remaining: 48s
12:	learn: 0.7624332	total: 633ms	remaining: 48.1s
13:	learn: 0.7558052	total: 684ms	remaining: 48.2s
14:	learn: 0.7490141	total: 735ms	remaining: 48.3s
15:	learn: 0.7429745	total: 791ms	remaining: 48.6s
16:	learn: 0.7370709	total: 843ms	remaining: 48.8s
17:	learn: 0.7311021	total: 894ms	remaining: 48.8s
18:	learn: 0.7254656	total: 945ms	remaining: 48.8s
19:	learn: 0.7199905	total: 998ms	remaining

In [60]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_CatBR), 2)
Q2_CV

0.6

In [61]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_CatBR)), 2)
RMSE_CV

0.57

# save the model to disk

In [62]:
pickle.dump(best_CatBR, open('Models/MACCS/Toxicity_CatBoost_MF.pkl', 'wb'))

# load the model from disk

In [17]:
best_CatBR = pickle.load(open('Models/MACCS/Toxicity_CatBoost_MF.pkl', 'rb'))